In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/spotifyclassifier-b2d573dee4ec.json"
from google.cloud import bigquery
# Construct a BigQuery client ob
client = bigquery.Client()

In [7]:
query_string = """
SELECT * from
spotifyclassifier.features.features_updated
"""

dataframe = (
    client.query(query_string)
    .result()
    .to_dataframe(
    )
)

Get pitch values for each song, split them into separate columns

In [9]:
import pandas as pd 
list_pitches = []
for i in range(len(dataframe)):
    pitches = []
    for j in dataframe['avg_pitches'][i]['list']:
        pitches.append(j['item'])
    list_pitches.append(pitches)
split_df = pd.DataFrame(list_pitches, columns=["pitch" + str(i) for i in range(12)])
split_df

,pitch0,pitch1,pitch2,pitch3,pitch4,pitch5,pitch6,pitch7,pitch8,pitch9,pitch10,pitch11
0,0.166429,0.130373,0.280209,0.397557,0.297497,0.488527,0.485772,0.277145,0.147838,0.123414,0.105160,0.123485
1,0.279721,0.212368,0.374908,0.213336,0.188126,0.162494,0.203560,0.463196,0.244858,0.282957,0.411426,0.225928
2,0.323854,0.253197,0.235020,0.235190,0.357220,0.240654,0.236476,0.247179,0.247417,0.361604,0.275023,0.281492
3,0.298196,0.338050,0.207071,0.297543,0.281848,0.368877,0.373803,0.373849,0.374974,0.227802,0.312116,0.227256
4,0.548001,0.413204,0.195386,0.253934,0.252862,0.267073,0.197820,0.208243,0.317224,0.229627,0.292882,0.305598
...,...,...,...,...,...,...,...,...,...,...,...,...
33109,0.566377,0.640027,0.692849,0.393384,0.323872,0.366166,0.371707,0.398733,0.302213,0.431418,0.288393,0.305517
33110,0.655814,0.693139,0.519790,0.396724,0.425710,0.466575,0.457470,0.504973,0.408510,0.499244,0.352426,0.273599
33111,0.315026,0.345199,0.276372,0.248277,0.367143,0.252540,0.305663,0.398445,0.383953,0.761932,0.328645,0.206549
33112,0.559399,0.743593,0.353880,0.339809,0.417672,0.419655,0.435183,0.391566,0.492129,0.390429,0.356983,0.326345


In [ ]:
dataframe = pd.concat([dataframe, split_df], axis=1)
dataframe = dataframe.drop('avg_pitches', axis=1)

Get timbre values for each song, split them into separate columns

In [12]:
list_timbre = []
for i in range(len(dataframe)):
    timbre = []
    for j in dataframe['avg_timbre'][i]['list']:
        timbre.append(j['item'])
    list_timbre.append(timbre)
split_df = pd.DataFrame(list_timbre, columns=["timbre" + str(i) for i in range(12)])
split_df

,timbre0,timbre1,timbre2,timbre3,timbre4,timbre5,timbre6,timbre7,timbre8,timbre9,timbre10,timbre11
0,37.004007,-90.234091,76.550343,8.322191,57.467659,6.286247,-26.871774,3.819632,-5.004314,-7.116997,-9.644667,-8.502539
1,39.728789,-46.906011,3.882081,-25.227432,42.089889,-6.057183,-19.772108,-0.730611,11.345934,-6.422826,-10.928428,13.846691
2,40.572993,-51.000179,8.265204,-26.386425,49.225155,-11.011279,-27.664425,1.049199,4.244098,-1.402933,-5.109533,8.181321
3,36.129289,56.935988,-16.474612,-0.480664,57.140670,-12.449146,-19.503267,-10.389212,11.565958,0.386632,-17.826369,-5.611879
4,40.551189,14.553666,22.092379,18.360686,17.393505,-16.239677,0.801369,2.288538,-3.734672,1.341475,-6.676309,14.716670
...,...,...,...,...,...,...,...,...,...,...,...,...
33109,53.737970,88.661980,23.983986,-12.633994,-2.576030,-34.321692,7.650654,0.721549,-6.285713,4.057271,-6.412553,0.494314
33110,54.245286,82.167680,10.401692,-12.709320,10.148679,-38.148919,5.099568,2.668915,1.629369,8.067857,-7.436330,-2.760537
33111,52.606308,41.804700,18.442318,-5.595646,22.383491,-26.831743,3.388637,-3.838996,-4.854613,1.800901,-9.741995,3.086852
33112,52.016924,63.030750,6.572731,-12.872339,-2.005959,-35.420363,9.631159,-1.574929,-0.732695,7.431930,-7.375142,2.246927


In [13]:
dataframe = pd.concat([dataframe, split_df], axis=1)
dataframe = dataframe.drop('avg_timbre', axis=1)
dataframe

,id,super_genre,subgenre,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_date,popularity,__index_level_0__,pitch0,pitch1,pitch2,pitch3,pitch4,pitch5,pitch6,pitch7,pitch8,pitch9,pitch10,pitch11,pitch0,pitch1,pitch2,pitch3,pitch4,pitch5,pitch6,pitch7,pitch8,pitch9,pitch10,pitch11,timbre0,timbre1,timbre2,timbre3,timbre4,timbre5,timbre6,timbre7,timbre8,timbre9,timbre10,timbre11
0,4VMVCovOM1xnOKxzr9Sje5,jazz,blues,0.529,0.0480,-16.170,0,0.0351,0.996000,0.000341,0.203,0.142,80.631,4,1923,0,3774,0.166429,0.130373,0.280209,0.397557,0.297497,0.488527,0.485772,0.277145,0.147838,0.123414,0.105160,0.123485,0.166429,0.130373,0.280209,0.397557,0.297497,0.488527,0.485772,0.277145,0.147838,0.123414,0.105160,0.123485,37.004007,-90.234091,76.550343,8.322191,57.467659,6.286247,-26.871774,3.819632,-5.004314,-7.116997,-9.644667,-8.502539
1,41CHb7F7SXcmkj0h8wekeF,jazz,jazz,0.484,0.0823,-11.273,0,0.0364,0.980000,0.000002,0.159,0.191,127.089,3,1933,49,20089,0.279721,0.212368,0.374908,0.213336,0.188126,0.162494,0.203560,0.463196,0.244858,0.282957,0.411426,0.225928,0.279721,0.212368,0.374908,0.213336,0.188126,0.162494,0.203560,0.463196,0.244858,0.282957,0.411426,0.225928,39.728789,-46.906011,3.882081,-25.227432,42.089889,-6.057183,-19.772108,-0.730611,11.345934,-6.422826,-10.928428,13.846691
2,0goLt4LXK1MpzwIfmItIS1,jazz,jazz,0.411,0.1360,-12.035,0,0.0359,0.985000,0.000000,0.100,0.138,75.090,4,1949,38,19881,0.323854,0.253197,0.235020,0.235190,0.357220,0.240654,0.236476,0.247179,0.247417,0.361604,0.275023,0.281492,0.323854,0.253197,0.235020,0.235190,0.357220,0.240654,0.236476,0.247179,0.247417,0.361604,0.275023,0.281492,40.572993,-51.000179,8.265204,-26.386425,49.225155,-11.011279,-27.664425,1.049199,4.244098,-1.402933,-5.109533,8.181321
3,7ij7UtqIKu2dP1qq5738rV,jazz,jazz,0.565,0.2300,-17.681,0,0.0768,0.734000,0.000713,0.105,0.669,78.712,4,1954,32,20312,0.298196,0.338050,0.207071,0.297543,0.281848,0.368877,0.373803,0.373849,0.374974,0.227802,0.312116,0.227256,0.298196,0.338050,0.207071,0.297543,0.281848,0.368877,0.373803,0.373849,0.374974,0.227802,0.312116,0.227256,36.129289,56.935988,-16.474612,-0.480664,57.140670,-12.449146,-19.503267,-10.389212,11.565958,0.386632,-17.826369,-5.611879
4,7wyznHrs1tQaeLMRdteaEh,jazz,jazz,0.718,0.3320,-13.213,0,0.0331,0.822000,0.892000,0.158,0.701,113.813,4,1954,0,20322,0.548001,0.413204,0.195386,0.253934,0.252862,0.267073,0.197820,0.208243,0.317224,0.229627,0.292882,0.305598,0.548001,0.413204,0.195386,0.253934,0.252862,0.267073,0.197820,0.208243,0.317224,0.229627,0.292882,0.305598,40.551189,14.553666,22.092379,18.360686,17.393505,-16.239677,0.801369,2.288538,-3.734672,1.341475,-6.676309,14.716670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33109,7dbjRLY969SvR0inx3h1eP,rock,rock,0.400,0.9680,-4.421,1,0.0769,0.000012,0.155000,0.106,0.434,144.054,4,2018,0,28954,0.566377,0.640027,0.692849,0.393384,0.323872,0.366166,0.371707,0.398733,0.302213,0.431418,0.288393,0.305517,0.566377,0.640027,0.692849,0.393384,0.323872,0.366166,0.371707,0.398733,0.302213,0.431418,0.288393,0.305517,53.737970,88.661980,23.983986,-12.633994,-2.576030,-34.321692,7.650654,0.721549,-6.285713,4.057271,-6.412553,0.494314
33110,7gQ7DfSSc3b8e4cHtFnDxu,rock,rock,0.478,0.9710,-2.729,1,0.0858,0.000038,0.000028,0.295,0.284,90.526,4,2018,64,28960,0.655814,0.693139,0.519790,0.396724,0.425710,0.466575,0.457470,0.504973,0.408510,0.499244,0.352426,0.273599,0.655814,0.693139,0.519790,0.396724,0.425710,0.466575,0.457470,0.504973,0.408510,0.499244,0.352426,0.273599,54.245286,82.167680,10.401692,-12.709320,10.148679,-38.148919,5.099568,2.668915,1.629369,8.067857,-7.436330,-2.760537
33111,7kJkTFmKYHrp9LT8w0oqpG,rock,rock,0.582,0.9360,-3.427,1,0.0382,0.048200,0.000009,0.341,0.372,102.019,4,2018,47,28962,0.315026,0.345199,0.276372,

Delete rows with more than 5 empty column values

In [ ]:
dataframe = dataframe.loc[dataframe.eq(0).sum(1).le(5),]
dataframe

Drop songs with multiple supergenre labels

In [ ]:
grouped_df = dataframe.groupby("id")
grouped_df = grouped_df.agg({"super_genre": "nunique"})
grouped_df = grouped_df.reset_index()
grouped_df = grouped_df[grouped_df['super_genre'] == 1]  
keep = grouped_df['id'].tolist()
dataframe = dataframe[dataframe['id'].isin(keep)]

Drop songs with multiple subgenre labels

In [ ]:
grouped_df = dataframe.groupby("id")
grouped_df = grouped_df.agg({"subgenre": "nunique"})
grouped_df = grouped_df.reset_index()
grouped_df = grouped_df[grouped_df['subgenre'] == 1]  
keep = grouped_df['id'].tolist()
dataframe = dataframe[dataframe['id'].isin(keep)]

Load dataframe into BigQuery table

In [ ]:
from google.cloud import bigquery
# Construct a BigQuery client object.
client = bigquery.Client()
table_id = "spotifyclassifier.features.features_updated_pit_tim_cols"

In [ ]:
job = client.load_table_from_dataframe(
    dataframe, table_id
)  # Make an API request.
job.result()  # Wait for the job to complete.

In [ ]:
table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)